# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather = pd.read_csv("../WeatherPy/Export/City_Weather.csv")
cities_weather = cities_weather.dropna(subset=['Country'])
display(cities_weather)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudliness,Wind Speed,Country,Date
0,0,Strezhevoy,60.732758,77.600807,25.59,77.0,0.0,1.99,RU,1.651379e+09
1,2,Port alfred,-33.598184,26.891477,56.05,83.0,1.0,1.99,ZA,1.651361e+09
2,3,Tura,64.274196,100.216964,23.56,75.0,97.0,4.18,RU,1.651379e+09
3,4,Hilo,19.707373,-155.081580,78.19,80.0,100.0,5.75,US,1.651318e+09
4,5,Itaituba,-4.262522,-55.987814,87.85,66.0,40.0,3.44,BR,1.651343e+09
...,...,...,...,...,...,...,...,...,...,...
552,600,Zaltan,32.946146,11.866804,64.17,78.0,3.0,24.92,LY,1.651362e+09
553,602,Tucuman,-26.830370,-65.203813,60.84,77.0,100.0,8.05,AR,1.651344e+09
554,603,Trapani,38.017432,12.515992,61.92,82.0,0.0,3.44,IT,1.651362e+09
555,604,Quelimane,-17.877510,36.890216,73.35,88.0,0.0,2.30,MZ,1.651362e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations = cities_weather[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidities = cities_weather["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
warm_cities = cities_weather[cities_weather["Max Temp"] > 70]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = warm_cities[["City", "Lat", "Lng", "Country"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Country,Hotel Name
3,Hilo,19.707373,-155.081580,US,
4,Itaituba,-4.262522,-55.987814,BR,
9,Bedford,32.844017,-97.143067,US,
10,Tarrafal,15.278416,-23.752294,CV,
11,Catemaco,18.421394,-95.111778,MX,
...,...,...,...,...,...
548,Bandar-e lengeh,26.562067,54.883817,IR,
549,Ambodifototra,-16.999638,49.852577,MG,
551,Lata,-10.724173,165.797764,SB,
555,Quelimane,-17.877510,36.890216,MZ,


In [12]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # find the closest hotel to coordinates
    coordinates = (str(row["Lat"]) + "," + str(row["Lng"]))
    #     coordinates = "11.4391193,124.5050324"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = coordinates
    
    print("Retrieving Results for " + row["City"])
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
#     print("Results " + str(results))

#     print("Results " + str(results[0]["name"]))
    try:
        print("Closest hotel is " + results[0]['name'])
#         print(results[0]['name'])
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
#         hotel_df.loc[index, 'address'] = results[0]['vicinity']
#         hotel_df.loc[index, 'price_level'] = results[0]['price_level']
#         hotel_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Hilo
Closest hotel is SCP Hilo Hotel
Retrieving Results for Itaituba
Closest hotel is Hotel Campos
Retrieving Results for Bedford
Closest hotel is Holiday Inn Dallas DFW Airport Area West
Retrieving Results for Tarrafal
Closest hotel is Hotel Ixola
Retrieving Results for Catemaco
Closest hotel is Hotel Playa Cristal
Retrieving Results for Lompoc
Closest hotel is Holiday Inn Express Lompoc, an IHG Hotel
Retrieving Results for Sakakah
Closest hotel is Fakhamat Al Orjoana Apartment
Retrieving Results for Bamnet narong
Closest hotel is Jong Jaroen Resort
Retrieving Results for Mattru
Closest hotel is Mondkor Town, Bagbo Chiefdom, Bo District
Retrieving Results for Jamestown
Closest hotel is Consulate
Retrieving Results for Marawi
Closest hotel is داخلية الشهيد وداعة الله
Retrieving Results for Bengkulu
Closest hotel is Grage Hotel Bengkulu
Retrieving Results for Belmonte
Closest hotel is Pousada Monte Carmelo O Pão
Retrieving Results for Kavaratti
Closest hotel is Ho

Missing field/result... skipping.
Retrieving Results for Kahului
Closest hotel is Maui Beach Hotel
Retrieving Results for Amarpur
Closest hotel is Md tanweer
Retrieving Results for Bathsheba
Closest hotel is Atlantis Hotel
Retrieving Results for Pahrump
Closest hotel is Best Western Pahrump Oasis
Retrieving Results for Ambilobe
Closest hotel is Hôtel & Restaurant DIANA Ambilobe
Retrieving Results for Lawton
Closest hotel is Holiday Inn Express & Suites Lawton-Fort Sill
Retrieving Results for Santa maria
Closest hotel is Porta do Vento
Retrieving Results for Lumberton
Closest hotel is Country Inn & Suites by Radisson, Lumberton, NC
Retrieving Results for Culaman
Closest hotel is Jasmin Ex House
Retrieving Results for Santa cruz cabralia
Closest hotel is Puerto Princesa Inn
Retrieving Results for Rajgarh
Closest hotel is Kanha Garden
Retrieving Results for Acapulco
Closest hotel is HS Hotsson Smart Acapulco
Retrieving Results for Vallenar
Closest hotel is Humacao Bed & Breakfast
Retrievi

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
# hotel_layer = gmaps.heatmap_layer(locations, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))